In [ ]:
'''
!pip3 install google-cloud-storage
!pip3 install google-cloud-bigquery
'''

In [1]:
from google.cloud import storage

# Path to your service account key file
path_to_private_key = 'is3107-418903-a6116d294003.json'

# Authenticate the session
storage_client = storage.Client.from_service_account_json(json_credentials_path=path_to_private_key)

## Uploading into Google Cloud Storage

In [2]:

'''
Uploads pandas DataFrame into Cloud Storage as CSV
category -> ["sgcarmrt", "motoristsg", "sgCOE", "carAPI"]
Ensure filename has a unique date identifier DDMMYYYY at the back
Return file path so can pass into the next transformation function
'''
def upload_to_storage(df, file_name, category):
    
    # Convert DataFrame to CSV string
    csv_string = df.to_csv(index=False)
    
    # Get the bucket object
    bucket_name = "is3107-datasets"
    bucket = storage_client.bucket(bucket_name)

    # Create a blob object from the bucket and assign it the file name
    file_path = f"{category}/{file_name}"
    blob = bucket.blob(file_path)
    
    # Upload the CSV string to GCS
    blob.upload_from_string(csv_string, 'text/csv')

    return file_path

In [43]:
'''
test upload into cloud storage
'''
from sgcarmart import run_test_scraper 

test_df = run_test_scraper(30)

Test scraper running...
--------------------------
Page 30 completed in 1.069312036037445m
--------------------------


In [44]:
test_df.head()

,name,price,depreciation,mileage,eng_cap,power,reg_date,coe_left,owners,omv,arf,accessories
0,Toyota Estima 2.4A Aeras Premium Moonroof,"55,800","23,180","85,437","2,362",125.0,24-Nov-2015,1yr 7mths 23days,1,"30,801","35,122","Sports rims, twin power sliding doors/Moonroof..."
1,Mercedes-Benz A-Class A200 Sport,"168,800","19,760","35,000","1,332",120.0,27-Dec-2021,7yrs 8mths 25days,1,"28,297","31,616","Rear aircon, sport rim, MBUX multimedia system..."
2,Nissan Qashqai 1.2A DIG-T,"71,500","14,410","75,500","1,197",85.0,29-Jun-2018,4yrs 2mths 28days,3,"20,452","20,633",Original leather seat. Keyless entry and start...
3,Porsche Macan S 3.0A PDK Panoramic Roof (New 1...,"190,800","19,070","129,000","2,997",250.0,20-Nov-2014,10yrs,3,"76,009","108,817","PCM with navigation, Bluetooth, front and rear..."
4,Mercedes-Benz GLA-Class GLA45 S AMG 4MATIC+ Plus,"298,000","32,850","17,000","1,991",310.0,21-Sep-2021,7yrs 5mths 20days,2,"73,803","104,846","Burmester, glass Sunroof, multibeam LED, 21"" A..."


In [45]:
file_path = upload_to_storage(test_df, "test_31032024.csv", "sgcarmart")

# Loading data into Big Query from Cloud Storage

In [46]:
import numpy as np
import re

def transform_sgcarmart_data(df):
    # remove any rows with null values
    df_cleaned = df.replace("NA", np.nan)
    df_cleaned = df_cleaned.dropna(how="any")
    df_cleaned[df_cleaned.isin([np.nan]).any(axis=1)]

    # change data types
    num_cols = ["price", "depreciation", "mileage", "owners", "omv", "arf", "eng_cap", "power"]
    df_cleaned["owners"] = df_cleaned["owners"].str.replace("More than 6", "6")
    for col in num_cols:
        df_cleaned[col] = df_cleaned[col].astype("string").str.replace(",","").astype("float")

    # remove parenthesis in car name
    df_cleaned["name"] = df_cleaned["name"].str.replace(r'\(([^)]+)\)', "", regex=True)
    df_cleaned.head()

    # convert coe_left to years
    def convert_to_years(x):
        
        years = re.search(r'(\d+)yrs', x)
        if years == None:
            years = 0
        else:
            years = int(years.group(1))
                    
        months = re.search(r'(\d+)mths', x)
        if months == None:
            months = 0
        else:
            months = int(months.group(1))
                    
        days = re.search(r'(\d+)days', x)
        if days == None:
            days = 0
        else:
            days = int(days.group(1))
        
        total_years = years + months/12 + days/365
        return round(total_years,2)

    df_cleaned["coe_left"] = df_cleaned["coe_left"].apply(convert_to_years)

    return df_cleaned

In [47]:
# Convert csv from Cloud Storage into pandas DataFrame and transform it

# This function is for transforming sgcarmart data 
def transform_sgcarmart_csv(file_path):
    bucket = storage_client.bucket("is3107-datasets")
    blob = bucket.blob(file_path)
    data = blob.download_as_bytes()
    df = pd.read_csv(BytesIO(data))
    df_transformed = transform_sgcarmart_data(df)

    return df_transformed

In [48]:
from google.cloud import bigquery

bigquery_client = bigquery.Client.from_service_account_json(json_credentials_path=path_to_private_key)

'''
Load the full transformed DataFrame into temp table in BQ
'''

def load_temp_bq_table(df, project_id, dataset_id, table_id, schema):
    job_config = bigquery.LoadJobConfig(
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
    schema=schema)

    # Load the DataFrame to the temporary table
    job = bigquery_client.load_table_from_dataframe(
        df,
        f"{project_id}.{dataset_id}.{table_id}",
        job_config=job_config)
    
    job.result()  # Wait for the job to complete

In [49]:
# Replace these variables with your specific details
# Example for sgcarmart

import pandas as pd
from io import BytesIO

df = transform_sgcarmart_csv(file_path)
project_id = 'is3107-418903'
dataset_id = 'temp'
table_id = 'sgcarmart-temp'
schema = [
    bigquery.SchemaField("name", "STRING"),
    bigquery.SchemaField("price", "FLOAT"),
    bigquery.SchemaField("depreciation", "FLOAT"),
    bigquery.SchemaField("mileage", "FLOAT"),
    bigquery.SchemaField("eng_cap", "FLOAT"),
    bigquery.SchemaField("power", "FLOAT"),
    bigquery.SchemaField("reg_date", "STRING"),
    bigquery.SchemaField("coe_left", "FLOAT"),
    bigquery.SchemaField("owners", "INTEGER"),
    bigquery.SchemaField("omv", "FLOAT"),
    bigquery.SchemaField("arf", "FLOAT"),
    bigquery.SchemaField("accessories", "STRING")
]

# Overwrite the BigQuery temp table with the CSV data
load_temp_bq_table(df, project_id, dataset_id, table_id, schema)

In [50]:
# Append only new rows from temp table to final table

def append_final_table(target_dataset_id, target_table_id, temp_dataset_id, temp_table_id):   
    project_id = 'is3107-418903'
    merge_query = f"""
    MERGE `{project_id}.{target_dataset_id}.{target_table_id}` final
    USING `{project_id}.{temp_dataset_id}.{temp_table_id}` temp
    ON final.`name` = temp.`name` and final.`reg_date` = temp.`reg_date`
    WHEN NOT MATCHED THEN
    INSERT ROW
    """
    
    # Run the merge query
    merge_job = bigquery_client.query(merge_query)
    merge_job.result() 

append_final_table("sgVehicles", "sgcarmart", "temp", "sgcarmart-temp")